# Print Metrics

## Functions

In [1]:
from pathlib import Path
import os
import pandas as pd

In [2]:
def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"[+] Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])


In [3]:
def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    print_final_metrics(df)


In [4]:
import pandas as pd

def compare_final_metrics(csv1_path, csv2_path):
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)

    last1 = df1.iloc[-1]
    last2 = df2.iloc[-1]

    metrics_to_compare = {
        "train/box_loss": "Box Loss (Train)",
        "train/cls_loss": "Cls Loss (Train)",
        "train/dfl_loss": "DFL Loss (Train)",
        "metrics/precision(B)": "Precision",
        "metrics/recall(B)": "Recall",
        "metrics/mAP50(B)": "mAP@0.5",
        "metrics/mAP50-95(B)": "mAP@0.5:0.95",
        "val/box_loss": "Box Loss (Val)",
        "val/cls_loss": "Cls Loss (Val)",
        "val/dfl_loss": "DFL Loss (Val)"
    }

    print("Changes in Metrics Before and After Retraning:\n")
    print(f"{'Metric':<25} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 65)

    for key, label in metrics_to_compare.items():
        val1 = last1[key]
        val2 = last2[key]
        diff = val2 - val1
        if abs(diff) > 1e-6:
            # If increase, color green; if decrease, color red
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red

            # Printing with colors
            print(f"{label:<25} {val1:<10.5f} {val2:<10.5f} {diff:<10.5f} {trend}")


In [5]:
import json

def compare_maps(json_path1, json_path2):
    with open(json_path1, 'r') as f1, open(json_path2, 'r') as f2:
        metrics1 = json.load(f1)
        metrics2 = json.load(f2)

    print("\nmAP@0.5:0.95 Differences Before and After Retraning:\n")
    print(f"{'Class':<15} {'Before':<10} {'After':<10} {'Diff':<10} {'Trend'}")
    print("-" * 60)

    for class_name in metrics1:
        map1 = metrics1[class_name].get("mAP@0.5:0.95", 0)
        map2 = metrics2.get(class_name, {}).get("mAP@0.5:0.95", 0)

        diff = map2 - map1
        if abs(diff) > 1e-6:
            if diff > 0:
                trend = f"\033[92m Increase\033[0m"  # Green for increase
            else:
                trend = f"\033[91m Decrease\033[0m"  # Red for decrease

            # Printing with colors
            print(f"{class_name:<15} {map1:<10.4f} {map2:<10.4f} {diff:<10.4f} {trend}")


In [6]:
import json
from ultralytics import YOLO

def load_yolo_model(model_path):
    return YOLO(model_path)

def run_model_validation(model, yaml):
    return model.val(data=yaml)

def extract_per_class_metrics(results):
    """
    Extracts mAP@0.5:0.95 per class from results.
    NOTE: Only mAP@0.5:0.95 is available via `results.box.maps`
    """
    per_class_metrics = {}
    if hasattr(results.box, 'maps') and results.box.maps is not None:
        maps = results.box.maps  # This is a NumPy array [num_classes]
        for i, name in results.names.items():
            per_class_metrics[name] = {
                "class_id": i,
                "mAP@0.5:0.95": round(float(maps[i]), 4)
            }
    else:
        print("[-] No per-class mAP@0.5:0.95 data found.")
    return per_class_metrics

def save_metrics_to_json(metrics, output_path):
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)
    print(f"[+] Saved per-class metrics to {output_path}")

def evaluate_and_save_metrics(model_path, yaml, output_json_path="per_class_metrics.json"):
    model = load_yolo_model(model_path)
    results = run_model_validation(model, yaml)
    metrics = extract_per_class_metrics(results)
    save_metrics_to_json(metrics, output_json_path)






In [7]:
import json

def print_per_class_metrics(json_path="per_class_metrics.json"):
    with open(json_path, "r") as f:
        metrics = json.load(f)
    
    print("Per-Class mAP@0.5:0.95 Metrics:\n")
    print(f"{'Class Name':<15} {'Class ID':<10} {'mAP@0.5:0.95':<15}")
    print("-" * 40)
    
    for name, data in metrics.items():
        print(f"{name:<15} {data['class_id']:<10} {data['mAP@0.5:0.95']:<15}")


## Calling Functions

### Before Retraning

In [8]:
yolov8 = './runs/train/yolov8'
print_csv_metrics(yolov8)

Found results.csv at: ./runs/train/yolov8\results.csv
Total number of epochs: 100

========== Final Training Metrics ==========
Training Loss: 1.890630
Precision: 0.567870
Recall: 0.301020
mAP@0.5: 0.316460
mAP@0.5:0.95: 0.202430

========== Final Validation Metrics ==========
Validation Loss: 5.499160


In [9]:
best_pt_path = find_best_model(yolov8)
# evaluate_and_save_metrics(best_pt_path, yaml="yolov8.yaml",output_json_path="per_class_metrics.json")

[+] Found best.pt at: runs\train\yolov8\weights\best.pt


In [10]:
# print_per_class_metrics("per_class_metrics.json")

### After Retraning

In [11]:
new_path = './runs/train/fine-tune-yolov8'
print_csv_metrics(new_path)

Found results.csv at: ./runs/train/fine-tune-yolov8\results.csv
Total number of epochs: 42

========== Final Training Metrics ==========
Training Loss: 2.061550
Precision: 0.528410
Recall: 0.517220
mAP@0.5: 0.544540
mAP@0.5:0.95: 0.400130

========== Final Validation Metrics ==========
Validation Loss: 2.650920


In [12]:
best_pt_path = find_best_model(new_path)
# evaluate_and_save_metrics(best_pt_path, yaml="yolo_retrain.yaml",output_json_path="per_class_metrics_retrain.json")

[+] Found best.pt at: runs\train\fine-tune-yolov8\weights\best.pt


In [13]:
# print_per_class_metrics("per_class_metrics_retrain.json")

### Compare both metrics

In [14]:
new_path = './runs/train/fine-tune-yolov8'
old_path = './runs/train/yolov8'

results_csv_path = find_results_csv(new_path)
results_csv_path_1 = find_results_csv(old_path)

compare_final_metrics(results_csv_path_1, results_csv_path)

Changes in Metrics Before and After Retraning:

Metric                    Before     After      Diff       Trend
-----------------------------------------------------------------
Box Loss (Train)          0.65540    0.70618    0.05078     Increase
Cls Loss (Train)          0.39123    0.46930    0.07807     Increase
DFL Loss (Train)          0.84400    0.88607    0.04207     Increase
Precision                 0.56787    0.52841    -0.03946    Decrease
Recall                    0.30102    0.51722    0.21620     Increase
mAP@0.5                   0.31646    0.54454    0.22808     Increase
mAP@0.5:0.95              0.20243    0.40013    0.19770     Increase
Box Loss (Val)            2.04225    0.78784    -1.25441    Decrease
Cls Loss (Val)            2.38927    0.85993    -1.52934    Decrease
DFL Loss (Val)            1.06764    1.00315    -0.06449    Decrease


In [15]:
compare_maps("per_class_metrics.json", "per_class_metrics_retrain.json")


mAP@0.5:0.95 Differences Before and After Retraning:

Class           Before     After      Diff       Trend
------------------------------------------------------------
unlabeled       0.2889     0.3403     0.0514      Increase
pool            0.8330     0.3403     -0.4927     Decrease
vegetation      0.2132     0.3403     0.1271      Increase
roof            0.4159     0.3041     -0.1118     Decrease
wall            0.2375     0.3723     0.1348      Increase
window          0.2343     0.0066     -0.2277     Decrease
person          0.2711     0.3375     0.0664      Increase
dog             0.4099     0.3403     -0.0696     Decrease
car             0.2429     0.3403     0.0974      Increase
bicycle         0.3350     0.3359     0.0009      Increase
tree            0.4122     0.5033     0.0911      Increase
truck           0.0360     0.3403     0.3043      Increase
bus             0.0000     0.3403     0.3403      Increase
vehicle         0.1146     0.5223     0.4077      Increase
